In [542]:
import numpy as np
import datetime as dt
import pandas as pd
import pickle
import copy
import pprint
import decimal
import statsmodels.formula.api as sm
from time_decorator import my_time_decorator
from ols import OLS
import math
import matplotlib.pyplot as plt
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [543]:
"""OLS model from statsmodels.api as sm
    .df_model # Degrees of Freedom of the Model
    .df_resid; # Degrees of Freedom of the Residuals
    model.endog_names
    model.exog_names

Deprecation Warning:
pandas_new.py:45: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
    DataFrame.rolling(center=False,window=5).mean()
      rolling_mean = pd.rolling_mean(price_table.iloc[::-1], window = 5).round(2)


Deleted lines of code that may be interesting:
filtered_data = daily_returns[np.isfinite(daily_returns['SPY'])]   .exog_names
price_table_reversed = price_table.iloc[::-1]
rolling_HVs = np.nanstd(daily_returns['SPY'])
rolling_mean = price_table.iloc[::-1].rolling(window=5).mean()
scrubbed.rename(index=str, columns={'AAPL': 'AAPL_scrubbed', 'SPY': 'SPY_scrubbed'}, inplace=True)
"""

"OLS model from statsmodels.api as sm\n    .df_model # Degrees of Freedom of the Model\n    .df_resid; # Degrees of Freedom of the Residuals\n    model.endog_names\n    model.exog_names\n\nDeprecation Warning:\npandas_new.py:45: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with \n    DataFrame.rolling(center=False,window=5).mean()\n      rolling_mean = pd.rolling_mean(price_table.iloc[::-1], window = 5).round(2)\n\n\nDeleted lines of code that may be interesting:\nfiltered_data = daily_returns[np.isfinite(daily_returns['SPY'])]   .exog_names\nprice_table_reversed = price_table.iloc[::-1]\nrolling_HVs = np.nanstd(daily_returns['SPY'])\nrolling_mean = price_table.iloc[::-1].rolling(window=5).mean()\nscrubbed.rename(index=str, columns={'AAPL': 'AAPL_scrubbed', 'SPY': 'SPY_scrubbed'}, inplace=True)\n"

In [544]:
# Import Price Table as Pandas DataFrame from Pickle File for S&P500 + Discretionary Symbols
price_table = pickle.load(open('sp500_price_table.pkl', 'rb')).head(20)[['AAPL', 'SPY']]
price_table.index = pd.to_datetime(daily_returns.index)
daily_returns = price_table / price_table.shift(-1) - 1
model = sm.OLS(daily_returns['AAPL'], daily_returns['SPY'], missing='drop')
results = model.fit()

In [545]:
# Calculate n-Day Rolling HVs.
rolling_HVs = daily_returns.iloc[::-1].rolling(window=5).std()*math.sqrt(252)

In [546]:
# Print Statements
# Head(500) returns 498 for results.df_resid. I want that number to be 499.
print("Beta: {:.2f}, Corr.: {:.2f}, n = {:.0f}".format(results.params['SPY'], results.rsquared, results.df_resid))

Beta: 0.84, Corr.: 0.56, n = 18


In [547]:
#daily_returns['AAPL_predicted'] = daily_returns.SPY*results.params['SPY']

In [548]:
initial_scrub = daily_returns[(abs(daily_returns.SPY) > .005) & (daily_returns.AAPL < .075)]
#initial_scrub.rename(index=str, columns={'AAPL': 'AAPL_initial_scrub', 'SPY': 'SPY_initial_scrub'}, inplace=True)
model = sm.OLS(initial_scrub['AAPL'], initial_scrub['SPY'], missing='drop')
results = model.fit()
initial_scrub['AAPL_predicted'] = initial_scrub.AAPL*results.params['SPY']
initial_scrub['error'] = initial_scrub.AAPL - initial_scrub.AAPL_predicted
initial_scrub['error_squared'] = initial_scrub.error*initial_scrub.error
initial_scrub

/home/paul/Environments/finance_env/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/paul/Environments/finance_env/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/paul/Environments/finance_env/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,AAPL,SPY,AAPL_predicted,error,error_squared
date,,,,,
2018-03-22,-0.014130,-0.024849,-0.011522,-0.002608,6.799212e-06
2018-03-19,-0.015279,-0.013530,-0.012460,-0.002820,7.950431e-06
2018-03-14,-0.008501,-0.005152,-0.006933,-0.001569,2.461348e-06
2018-03-13,-0.009630,-0.006453,-0.007853,-0.001777,3.158356e-06
2018-03-09,0.017181,0.017399,0.014010,0.003171,1.005277e-05
2018-03-05,0.003462,0.011529,0.002823,0.000639,4.081211e-07
2018-03-02,0.006914,0.005176,0.005638,0.001276,1.628115e-06
2018-03-01,-0.017516,-0.014562,-0.014284,-0.003232,1.044899e-05
2018-02-28,-0.001514,-0.010134,-0.001234,-0.000279,7.801480e-08


In [549]:
pct_cutoff = .15
second_scrub = initial_scrub[initial_scrub['error_squared'] < initial_scrub.error_squared.quantile(1-pct_cutoff)].loc[:, ['AAPL', 'SPY']]
second_scrub

,AAPL,SPY
date,,
2018-03-22,-0.014130,-0.024849
2018-03-19,-0.015279,-0.013530
2018-03-14,-0.008501,-0.005152
2018-03-13,-0.009630,-0.006453
2018-03-09,0.017181,0.017399
2018-03-05,0.003462,0.011529
2018-03-02,0.006914,0.005176
2018-02-28,-0.001514,-0.010134
2018-02-27,-0.003241,-0.012464


In [550]:
main = daily_returns.join(initial_scrub)
main

ValueError: columns overlap but no suffix specified: Index(['AAPL', 'SPY'], dtype='object')